In [1]:
%load_ext autoreload
%autoreload 2

# Vector Space retrieval with inverted files


## Helper functions

### Tokenizer & Set of Words

In [2]:
from utils import analyzer

print(analyzer.bag_of_words("this is a simple test for this function", remove_stopwords = True))
print(analyzer.bag_of_words("this is a simple test for this function", remove_stopwords = False))

{'function': 1, 'simple': 1, 'test': 1}
{'a': 1, 'for': 1, 'function': 1, 'is': 1, 'simple': 1, 'test': 1, 'this': 2}


### TopKList class

In [3]:
from heapq import heappop, heappush
from typing import Callable

class TopKList:
    """
        Maintains a list of top-k documents. Initializer accepts
        a list of tuples (term, weight) to provide information about
        weights used by retrieval model. Implements the iter() interface.
        Takes an optional predicate(doc_id: int) function to filter documents
        before returning them. 
    """
    def __init__(self, k: int, term_weights: list[tuple[str,float]] = None, predicate: Callable[[int], bool] = None):
        self.docs_heap = []
        self.k = k
        self.predicate = predicate
        self.results = []
        if term_weights:
            self.term_weights = term_weights
            self.terms = [term for term, _ in self.term_weights]
            self.weights = dict(self.term_weights)
    
    def add(self, doc_id: int, score: float):
        heappush(self.docs_heap, (-score, doc_id, {'id': doc_id, 'score': score}))
        # optional (infrequent) pruning if heap grows too large

    def __iter__(self):
        # do we already have the results?
        for entry in self.results:
            yield entry
        # produce more results (if necessary and available)
        rank = len(self.results)
        while rank < self.k and len(self.docs_heap) > 0:
            entry = heappop(self.docs_heap)[2]
            if self.predicate == None or self.predicate(entry['id']):
                rank += 1
                entry['rank'] = rank
                self.results.append(entry)
                yield entry

### IDF implementations and BM25 parameters
BM25 parameters are typically `k=1.2` and `b=0.75`, while `adl` must be set from the collection. If we leave `adl=None`, the term normalization does not take document length into account (which is ok if documents in the collection have about equal length)

In [4]:
import math
BM25 = { 'k': 1.2, 'b': 0.75, 'adl': None }

def idf(doc_freq: int, num_docs: int) -> float:
    return math.log((num_docs + 1) / (doc_freq + 1))

def idf_bm25(doc_freq: int, num_docs: int) -> float:
    return math.log((num_docs - doc_freq + 0.5) / (doc_freq + 0.5))
    
def idf_bm25_pos(doc_freq: int, num_docs: int) -> float:
    return math.log((num_docs + 1) / (doc_freq + 0.5))

### TF normalization functions
We apply document normalization at index building time. We also use normalized query vectors so that similarity becomes a simple dot product between document and query vector. The function below performs term normalization for documents given a bag-of-word and a vocabulary. The vocabulary maps a term to a dictionary that holds the idf values for the dot-product and cosine measure. 

In [5]:
import math

def normalize_doc_vector(vector: dict[str, int], vocabulary: dict[str, dict], measure: str) -> dict[str, float]:
    # dot-product: multiply each term's tf by its idf
    if measure == 'dot':
        return {term: tf * vocabulary[term]['idf'] for term,tf in vector.items()}

    # cosine-measure: multiply each term's tf by its idf and divide by total vector length
    if measure == 'cosine':
        norm = sum([(tf * vocabulary[term]['idf']) ** 2 for term, tf in vector.items()]) ** 0.5
        return {term: tf * vocabulary[term]['idf'] / norm for term, tf in vector.items()}

    # bm25: normalize with bm25 formula with document length
    if measure == 'bm25' and BM25['adl']:
        doc_len = sum(vector.values())
        return {term: tf * (BM25['k'] + 1) / (tf + BM25['k'] * (1 - BM25['b'] + BM25['b'] * doc_len / BM25['adl']))  for term, tf in vector.items()}

    # bm25: normalize with bm25 formula without document length
    if measure in ['bm25', 'bm25-nolen', 'bm25-pos']:
        return {term: tf * (BM25['k'] + 1) / (tf + BM25['k'])  for term, tf in vector.items()}

    raise ValueError('Unknown normalization measure')

## Vector Space Model Implementation

### The Base Retriever Class
* `n_docs: int`: number of documents added to index
* `documents dict[int, dict{'id', 'vector', 'norm-vector'}]`: collection of documents as dictionary with doc_id as key. Each document is a dictionary with the properties from the dataset and additional properties for the retrieval
  - `id` hold the document id as generated when loading the document; corresponds to the key in documents
  - `vector` holds the term freqeuncies as dictionary (key=term, value=term frequency)
  - `norm-vector` normalized vector for the selected measure
* `vocabulary: dict[term, dict{df, idf}]`: vocabulary with all terms. Values contain objects with document frequency and idf for selected measure
* `index: dict[term, list[tuple[int, int]]]`: inverted index mapping terms to postings. Postings contain doc_id and term frequency sorted by doc_id
  
The vector space model support 5 measures: `dot`, `cosine`, `bm25`, `bm25-nolen`, `bm25-pos`. The index needs to be rebuilt if the measure is changed (we normalize all documents and the measure imapcts normalization)

In [6]:
import math
class VSModel:
    """
        Generic class for the evaluation of the vector space model, inherited by the document-at-a-time (DAAT) and 
        term-at-a-time (TAAT) models. 
    """
    def __init__(self, collection: list[dict] = None, measure: str = 'dot', remove_stopwords: bool = True):
        self.build_index(collection or [], measure, remove_stopwords)
    
    def _add_document(self, doc: dict):
        self.n_docs += 1
        doc_id = doc['id'] = self.n_docs
        self.documents[doc_id] = doc
        # create vector from str-properties
        text = ' '.join([value for key, value in doc.items() if type(value) == str])
        doc['vector'] = analyzer.bag_of_words(text, remove_stopwords = self.remove_stopwords)
        doc['len'] = sum(doc['vector'].values())
        # add to vocabulary and count document frequency
        for term, tf in doc['vector'].items():
            self.vocabulary[term] = self.vocabulary.get(term, 0) + 1
    
    def _build_vocabulary(self):
        idf_func = self.measure in ['bm25', 'bm25-nolen', 'bm25-pos'] and idf_bm25 or idf
        self.vocabulary = dict([(term, {'df': df, 'idf': idf_func(df, self.n_docs)}) for term, df in self.vocabulary.items()])

    def _normalize_vectors(self):
        BM25['adl'] = sum([doc['len'] for doc in self.documents.values()]) / self.n_docs
        for doc_id, doc in self.documents.items():
            doc['norm-vector'] = normalize_doc_vector(doc['vector'], self.vocabulary, self.measure)

    def _build_postings(self):
        for doc_id, doc in self.documents.items():
            for term, tf_norm in doc['norm-vector'].items():
                self.index.setdefault(term, []).append((doc_id, tf_norm))

    def build_index(self, collection: list[dict], measure: str = 'dot', remove_stopwords: bool = True):
        self.remove_stopwords = remove_stopwords
        self.measure = measure
        self.n_docs = 0
        self.doc_len_sum = 0
        self.documents = {}
        self.index = {}
        self.vocabulary = {}
        # load all documents
        for doc in collection:
            self._add_document(doc)
        # finalize the index with idf, normalization, and the postings
        if self.n_docs:
            self._build_vocabulary()
            self._normalize_vectors()
            self._build_postings()

In [7]:
class VSModel(VSModel):
    """
        Generic class for the evaluation of the Vector Space model, inherited by the document-at-a-time (DAAT) and 
        term-at-a-time (TAAT) implementation. This superclass defines the idf-weights including filtering the most
        important terms.
    """
    def query_weights(self, vector: dict[str, int], measure: str) -> list[tuple[str,float]]:
        # remove terms not in vocabulary
        terms = list(filter(lambda t: t in self.vocabulary, vector.keys()))
        # dot product: multiply tf with idf
        if measure == 'dot':
            return list(map(lambda t: (t, vector[t] * self.vocabulary[t]['idf']), terms))
        # cosine measure: multiply tf with idf and take the cosine of the sum  
        if measure == 'cosine':
            norm = sum([(tf * self.vocabulary[term]['idf']) ** 2 for term, tf in vector.items()]) ** 0.5
            return list(map(lambda t: (t, vector[t] * self.vocabulary[t]['idf'] / norm), terms))
        # bm25: ignore tf and just use idf of term as weight
        if measure in ['bm25', 'bm25-nolen', 'bm25-pos']:
            return list(map(lambda t: (t, self.vocabulary[t]['idf']), terms))
     
        raise ValueError('Unknown normalization measure')

### Document-at-a-time for Vector Space Model

In [8]:
class VSModel_DAAT(VSModel):
    """
        Implements the DAAT model for the Vector Space model using inverted index method.
    """
    def search(self, query: str, k: int, measure: str = 'dot', predicate: Callable[[int], bool] = None, selected_docs: set[int] = None):
        query_vector = analyzer.bag_of_words(query)

        # filter terms and obtain c_j-weights for terms in order of their importance 
        term_weights = self.query_weights(query_vector, measure)

        # get iterators for each term and fetch first posting; postings have form (term, tf)
        iters = [iter(self.index[term]) for (term, _) in term_weights]
        nexts = [next(iter, None) for iter in iters]

        # keep track of all retrieved documents and their score; stored as tuples (doc_id, score)
        topk = TopKList(k, term_weights, predicate)

        # iterate through all streams and calculate score for smallest doc id
        while not all(e is None for e in nexts):
            # get smallest value from nexts, ignoring None values
            smallest = min(nexts, key = lambda x: x and x[0] or math.inf)[0]
            # if selected_docs is given and smallest is not in selected_docs, skip this document
            if selected_docs == None or smallest in selected_docs:
                # if so, add it to topk
                score = sum([nexts[i][1] * term_weights[i][1] for i in range(len(nexts)) if nexts[i] and nexts[i][0] == smallest])
                topk.add(smallest, score)
            # for each entry in nexts, fetch next item if entry equals smallest
            for i, e in enumerate(nexts):
                if e and e[0] == smallest:
                    nexts[i] = next(iters[i], None)
        
        # finsihed, return topk for result iteration
        return topk

### Term-at-a-time for Vector Space Model

## Running some examples

### Loading the data

In [9]:
import ipywidgets as widgets
opt_strategy = widgets.Dropdown(options=['document-at-a-time', 'term-at-a-time'])
opt_dataset = widgets.Dropdown(options=['random', 'imdb movies'])
opt_measure = widgets.Dropdown(options=['dot', 'cosine', 'bm25', 'bm25-pos', 'bm25-nolen'])
display(widgets.HBox([opt_strategy, opt_dataset, opt_measure]))

In [10]:
from utils import table
from datasets import random as random_docs, imdb as imdb_docs
import random

# select the strategy of the retrieval model
if opt_strategy.value == 'document-at-a-time':
    retriever = VSModel_DAAT()
else:
    retriever = VSModel_TAAT()

# select the dataset and define feedback function, queries, predicates, and selections
if opt_dataset.value == 'random':
    collection = random_docs
    assessments = {
        'random': lambda id: random.random() < 0.8,
        'id < 20': lambda id: id < 20,
    }
    queries = [
        'cat dog',
        'horse bird',
        'cat dog horse bird'
    ]
    predicates = {
        'even doc ids': lambda id: id % 2 == 0,
        'odd doc ids': lambda id: id % 2 == 1,
    }
    selections = {
        'doc<10': list(range(10)),
    }
elif opt_dataset.value == 'imdb movies':
    collection = imdb_docs
    assessments = {
        'top-100': lambda id: id < 100,
        'star in title': lambda id: 'star' in retriever.documents[id]['title'].lower(),
        'morgan in actor': lambda id: 'morgan' in retriever.documents[id]['actors'].lower(),
        'comedy in genre': lambda id: 'comedy' in retriever.documents[id]['genre'].lower(),
    }
    queries = [
        'star wars', 
        'drama morgan freeman', 
        'comedy'
    ]
    predicates = {
        'year < 1990': lambda id: retriever.documents[id]['year'] < 1990,
        'year >= 1990': lambda id: retriever.documents[id]['year'] >= 1990,
    }
    selections = {
        'top-100': list(range(100)),
        'top-250': list(range(250)),
    }
else:
    raise ValueError("to be implemented")

# build index
retriever.build_index(collection.load(), opt_measure.value)

### Inspecting the data

In [11]:
table.print([collection.format(doc) for doc in retriever.documents.values()], collection.headers(), max_rows = 10)
table.print(sorted([[term, term_data['df'], round(term_data['idf'], 2), ', '.join([f'{doc_id} ({round(w, 2)})' for doc_id, w in retriever.index[term]])] for term, term_data in retriever.vocabulary.items()], key=lambda x: -x[1]), ['term', 'df', 'idf', 'posting'], max_rows=20)

print(f'{len(retriever.documents)} documents in collection')
print(f'{len(retriever.vocabulary)} distinct terms in collection')
print('{count} postings'.format(count=sum([len(postings) for postings in retriever.index.values()])))

|   id | text                                                                                                     |
|------|----------------------------------------------------------------------------------------------------------|
|    1 | horse bear tiger fly fly fly fly                                                                         |
|    2 | dog dog dog dog ostrich ostrich ostrich ant                                                              |
|    3 | dog dog dog dog dog bird bird bird donkey donkey donkey donkey donkey                                    |
|    4 | dog dog dog cat                                                                                          |
|    5 | horse horse horse horse rabit rabit lion lion lion lion lion bee bee bee bee fly fly fly fly snake snake |
|    6 | ostrich ostrich fly                                                                                      |
|    7 | bird bird bird snake snake snake snake                         

### Pretty printing functions

In [12]:
def print_topk(topk: TopKList):
    list = []
    for entry in topk:
        list.append(collection.format(retriever.documents[entry['id']], [
            entry['rank'],
            round(entry['score'], 2)
        ]))
    table.print(list, collection.headers('rel', 'score'), max_rows=len(list))

### Searching with selected data set and similarity measure

In [13]:
from IPython.display import clear_output
from functools import reduce

def run_query(query: str, k:int, predicate: str, selection: str):
    topk = retriever.search(query, k, measure=opt_measure.value, predicate=predicates.get(predicate, None), selected_docs=selections.get(selection, None))
    print_topk(topk)
    for term in sorted(topk.weights.keys(), key = lambda term: -topk.weights[term]):
        print(term.rjust(16), topk.weights[term])

# display query form
widgets.interact(run_query,
    query=widgets.Dropdown(description='query', options=list(queries)),
    k=widgets.IntSlider(min=5, max=50, step=5, value=20),
    predicate=widgets.Dropdown(description='predicate',options=['<none>'] + list(predicates.keys())),
    selection=widgets.Dropdown(description='selection',options=['<none>'] + list(selections.keys())),
);

interactive(children=(Dropdown(description='query', options=('cat dog', 'horse bird', 'cat dog horse bird'), v…

In [14]:
retriever.documents[235]

KeyError: 235

## What's next?

## Building the index from document collection

In [ ]:
from utils import stopwords
from itertools import groupby
import re

def tokenize(text: str) -> list[str]:
    text = re.sub(r'[,\.\-\?!\(\)\s:;_\'"\+\*\&\$]', ' ', text.lower())
    text = re.sub(r'\s+', ' ', text).strip()
    return filter(lambda t: t not in stopwords.english, text.split(' '))

def bag_of_words(tokens: list[str]) -> dict[str, int]:
    return dict([(token, len(list(group))) for token, group in groupby(sorted(tokens))])

def add_document(doc: dict):
    global nDocs, index, documents, vocabulary
    nDocs += 1
    # add document to documents dict, with id as key and all properties
    doc_id = doc['id'] = nDocs
    documents[doc_id] = doc
    # get terms from all string properties of doc
    text = ' '.join([value for key, value in doc.items() if type(value) == str])
    doc['vector'] = bag_of_words(tokenize(text))
    doc['len'] = sum([tf for _, tf in doc['vector'].items()])
    # update vocabulary (holding df values during this loop) and index (term -> postings)
    for term, tf in doc['vector'].items():
        vocabulary[term] = vocabulary.get(term, 0) + 1
        index[term] = index.get(term, []) + [(doc_id, tf)]

def build_index(collection: list[dict]):
    global nDocs, index, documents, vocabulary
    nDocs = 0
    index = {}
    documents = {}
    vocabulary = {}
    for doc in collection:
        add_document(doc)
    # update vocabulary -> create dict with df, idf, and idf_bm25 values
    vocabulary = dict([(term, {'df': df, 'idf': idf(df, nDocs), 'idf_bm25': idf_bm25(df, nDocs)}) for term, df in vocabulary.items()])
    # calculate norm of vector for cosine measure
    for doc in documents.values():
        doc['norm'] = sum([(tf * vocabulary[term]['idf']) ** 2 for term, tf in doc['vector'].items()]) ** 0.5      

## IMDB Search Example
### Load the data set

In [ ]:
# loading the imdb data set (1000 movies)
from datasets.docs import imdb
from utils import stopwords

data = imdb.load()
def doc_format_imdb(doc: dict) -> str:
    trim = lambda s,n: len(s) > n and s[:n] + "\u2026" or s
    title_ex = '{title_short} ({year}, {runtime}m, {rating})'.format(title_short=trim(doc['title'], 30), **doc)
    return '{title_ex:<50} {genre_short:<20} {summary} [{actors}]'.format(title_ex=title_ex, genre_short=trim(doc['genre'], 18), **doc)

for item in data[:5]:
    print(doc_format_imdb(item))

data[0]

In [ ]:
build_index(data)
print('{count} documents in collection'.format(count=nDocs))
print('{count} distinct terms in collection'.format(count=len(vocabulary)))
print('{count} postings'.format(count=sum([len(postings) for postings in index.values()])))

### Looking at top (and some random) terms in the vocabulary including postings from the index

In [ ]:
# print vocabulary with df and idf
print('term               df     idf idf_bm25    postings')
print('-' * 100)
for term, item in sorted(vocabulary.items(), key=lambda t: -t[1]['df'])[:10]:
    print("{term:16} {df:>4d} {idf:>7.2f} {idf_bm25:>8.2f}    {postings}".format(term=term.ljust(10), df=item['df'], idf=item['idf'], idf_bm25=item['idf_bm25'], postings=index[term]))
print()
for term, item in random.sample(list(vocabulary.items()), 15):
    print("{term:16} {df:>4d} {idf:>7.2f} {idf_bm25:>8.2f}    {postings}".format(term=term.ljust(10), df=item['df'], idf=item['idf'], idf_bm25=item['idf_bm25'], postings=index[term]))

### Looking at some random documents

In [ ]:
# print a few documents
print('  id |  len     norm   vector')
print('-' * 100)
for doc_id in random.sample(range(1, len(documents) + 1), 25):
    print("{id:>4} | {len:>4d} {norm:>8.2f}   {vector}".format(id=doc_id, len=documents[doc_id]['len'], norm=documents[doc_id]['norm'], vector=str(documents[doc_id]['vector'])))

### Queries

In [ ]:
from typing import Callable
def search(query: str, k: int, measure: str, predicate: str, algorithm: str) -> TopKList:
    query_vector = bag_of_words(tokenize(query))
    vsmodel = {'ddat': VSModel_DAAT}.get(algorithm.lower(), VSModel_DAAT)
    predicate_func = {
        'year > 1990 (predicate)': lambda doc_id: documents[doc_id]['year'] > 1990, 
        'year <= 1990 (predicate)': lambda doc_id: documents[doc_id]['year'] <= 1990, 
    }.get(predicate, None)
    selected_docs = {
        'top-100 (selection)': set(range(1,101))
    }.get(predicate, None)
    topk = vsmodel.search(query_vector, k, measure, predicate=predicate_func, selected_docs=selected_docs)
    print_topk(topk, doc_format_imdb)

# options for the dialog
queries = ['star wars', 'drama morgan freeman', 'comedy']
measures = ['cosine', 'dot', 'bm25']
predicates = ['<none>', 'year > 1990 (predicate)', 'year <= 1990 (predicate)', 'top-100 (selection)']
algorithm = ['ddat', 'taat']

# interactive selection of scenario
widgets.interact(search, 
    query=widgets.Dropdown(options=queries), 
    k=widgets.IntSlider(min=5, max=50, step=5, value=20),
    measure=widgets.Dropdown(options=measures),
    predicate=widgets.Dropdown(options=predicates),
    algorithm=widgets.Dropdown(options=algorithm),
);


In [ ]:
queries = ['star wars', 'drama morgan freeman', 'comedy']
print(list(enumerate(queries)))
q_dropdown = widgets.Dropdown(
    options=[('One', 1), ('Two', 2), ('Three', 3)],
    value=2,
    description='Number:',
)

In [ ]:
DEBUG = False

# initial step for "bird(2) horse"
query = {'bird': 2, 'horse':1}
k = 10
print(query)

# (optional) enable a predicate for the filtering step
predicate = None
# predicate = lambda doc_id: doc_id % 2 == 0
# predicate = lambda doc_id: doc_id % 2 == 1
selected_docs = None
# selected_docs = list(range(10))

# run query, display result, and get feedback
topk = VSModel_DAAT.query(query, k, 'cosine', predicate)
print_topk(topk)